# Debug notebook
This notebook is only used to troubleshoot the S1 ARD workflow processor.

## S1 ARD workflow for scenes overlapping AM

In [1]:
import timeout_decorator
import json
from utils.prepS1AM import prepareS1AM

@timeout_decorator.timeout(10800, use_signals=False)
def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    logger.info(f"{x}: {loaded_json[x]}")
    prepareS1AM(**loaded_json)

## S1 ARD job queue processor

In [2]:
import os
import logging
import rediswq

level = os.getenv("LOGLEVEL", "INFO").upper()
logging.basicConfig(format="%(asctime)s %(levelname)-8s %(name)s %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=level)

host = os.getenv("REDIS_SERVICE_HOST", "redis-master")
q = rediswq.RedisWQ(name="jobS1AM", host=host)

logger = logging.getLogger("worker")
logger.info(f"Worker with sessionID: {q.sessionID()}")
logger.info(f"Initial queue state: empty={q.empty()}")

2020-10-19 08:22:01 INFO     worker Worker with sessionID: d0425aea-3e7e-4329-a4fd-020279d8e8a1
2020-10-19 08:22:01 INFO     worker Initial queue state: empty=False


In [3]:
import datetime

while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        logger.info(f"Working on {itemstr}")
        start = datetime.datetime.now().replace(microsecond=0)

        # In case the COG conversion gets stuck, a TimeoutError is raised and we try again: the existance of a COG from an earlier iteration is often enough to progress upon retrying
        for x in range(0, 2):  # try 2 times
            e = False
            try:
                process_scene(itemstr)
                e = True
            except timeout_decorator.TimeoutError:
                logger.info(f"Timed out while working on {itemstr}")
                e = False
                pass
            if e:
                break

        q.complete(item)

        end = datetime.datetime.now().replace(microsecond=0)
        logger.info(f"Total processing time {end - start}")
    else:
        logger.info("Waiting for work")

2020-10-19 08:23:00 INFO     worker Working on {"in_scene": "S1A_IW_GRDH_1SSV_20161215T173221_20161215T173249_014393_017542_45B6", "s3_bucket": "public-eo-data", "s3_dir": "luigi/fiji/sentinel_1/"}
2020-10-19 08:23:00 INFO     root S1A_IW_GRDH_1SSV_20161215T173221_20161215T173249_014393_017542_45B6.SAFE S1A_IW_GRDH_1SSV_20161215T173221 Starting
2020-10-19 08:23:00 INFO     root S1A_IW_GRDH_1SSV_20161215T173221_20161215T173249_014393_017542_45B6.SAFE S1A_IW_GRDH_1SSV_20161215T173221 DOWNLOADING via ASF
2020-10-19 08:23:00 INFO     root fetching: https://api.daac.asf.alaska.edu/services/search/param?granule_list=S1A_IW_GRDH_1SSV_20161215T173221_20161215T173249_014393_017542_45B6&output=csv
2020-10-19 08:23:01 INFO     root logging into asf
2020-10-19 08:23:04 INFO     root successfully logged into asf
2020-10-19 08:23:04 INFO     root S1A_IW_GRDH_1SSV_20161215T173221_20161215T173249_014393_017542_45B6.SAFE S1A_IW_GRDH_1SSV_20161215T173221 DOWNLOADED via ASF
2020-10-19 08:23:04 INFO     r

not boo


2020-10-19 12:18:38 INFO     worker Total processing time 3:55:38


In [4]:
print("Queue empty!")

Queue empty!


---